In [21]:
import pandas as pd

df = pd.read_json('pyspam_features_with_offline.jsonl', lines = True)
pd.set_option('display.max_columns', None)
df.head()

,pkg_name,n_name_len,has_digit_in_name,has_dash_or_underscore,cat_name_case,n_summary_len,n_desc_len,n_desc_lines,has_code_block_in_desc,n_urls_in_desc,has_suspicious_kw,pct_non_ascii_desc,t_age_first_release_days,t_age_last_release_days,n_versions,t_median_release_gap_days,has_single_release,t_time_of_day_bucket,cat_weekday_of_last_release,n_maintainers,pct_free_email_domains,has_disposable_email,has_missing_author,has_homepage,has_repo_url,cat_repo_host,has_issue_tracker,has_docs_url,has_license,cat_license_family,n_classifiers,has_prog_lang_classifier,has_typing_classifier,n_distributions,n_requires,has_extras,rule_no_repo_low_desc_len,rule_suspicious_name_and_dep,versions,t_first_release,t_last_release,latest_project_urls,distinct_authors,distinct_maintainers,distinct_keywords,distinct_classifiers,latest_dependencies,is_spam,n_downloads_7d,n_downloads_30d,n_dependents_est,licenses,latest_summary,latest_description,n_lev_dist_to_top1,n_lev_dist_to_alias,sim_tfidf_to_legit_centroid,dist_embed_to_legit_desc,n_pkgs_by_maintainers_30d,n_low_download_pkgs_by_maintainers,has_dependency_to_top_brand,min_dep_lev_to_brand,has_dependency_lev_close_to_brand
0,aowdjpawojd,11,False,False,lower,0,0,1,False,0,False,0.0,1119,1119,1,0.0,True,evening_18_23,Saturday,0,0.0,0,0,0,0,unknown,0,0,0,other_or_unknown,0,0,0,2,0,0,1,0,"0.0.0,0.0.0",2022-11-05 23:04:36.607111 UTC,2022-11-05 23:04:38.050281 UTC,[],[],[],[],[],[],1,NaN,NaN,NaN,None,None,None,8,8,0.014934,1.000000,0,0,0,NaN,0
1,array-record,12,False,True,lower,59,0,1,False,0,False,0.0,1101,15,16,0.0,False,afternoon_12_17,Thursday,1,0.0,0,0,1,1,github,0,0,1,other_or_unknown,4,1,0,12,8,1,0,0,"0.8.0,0.8.1,0.8.1,0.8.0,0.8.0,0.8.1,0.8.1,0.8....",2022-11-23 21:52:08.633393 UTC,2025-11-13 16:15:45.092361 UTC,[],[no-reply@google.com],[],[],"[Programming Language :: Python :: 3.11, Progr...","[absl-py, etils[epath], apache-beam[gcp]>=2.53...",0,367475.0,1878537.0,3.0,Apache-2.0,A file format that achieves a new frontier of ...,None,10,9,0.069909,1.000000,1,0,1,0.0,1
2,azure-mgmt-loganalytics,23,False,True,lower,66,19482,489,True,11,False,0.0,2949,388,23,0.0,False,night_00_05,Tuesday,1,0.0,0,0,1,1,github,0,0,1,MIT,10,1,0,2,4,0,0,0,"13.0.0b4,9.0.0,13.0.0b4,9.0.0,13.0.0b3,13.0.0b...",2017-11-01 17:26:20.328161 UTC,2024-11-05 03:11:34.319759 UTC,[],[azpysdkhelp@microsoft.com],[],"[azure,azure sdk, azure, azure sdk]","[Programming Language :: Python, Programming L...","[isodate>=0.6.1, typing-extensions>=4.6.0, azu...",0,1714703.0,7594735.0,2.0,MIT License,Microsoft Azure Log Analytics Management Clien...,# Microsoft Azure SDK for Python\n\nThis is th...,17,19,0.153751,0.842534,18,0,0,5.0,0
3,azure-mgmt-notificationhubs,27,False,True,lower,70,13658,294,True,12,False,0.0,3570,617,16,0.0,False,morning_06_11,Thursday,2,0.0,0,0,1,1,github,0,0,1,MIT,10,1,0,2,3,0,0,0,"2.1.0,2.1.0,1.0.0,1.0.0,0.30.0rc5,0.30.0rc5,0....",2016-02-19 02:48:46.935974 UTC,2024-03-21 06:30:29.296433 UTC,[],"[azpysdkhelp@microsoft.com, ptvshelp@microsoft...",[],"[azure, azure sdk, azure,azure sdk]","[Programming Language :: Python :: 3, Programm...","[isodate>=0.6.1, azure-common>=1.1, azure-mgmt...",0,342109.0,1511511.0,1.0,"Apache License 2.0, MIT License",Microsoft Azure Notification Hubs Management C...,# Microsoft Azure SDK for Python\n\nThis is th...,21,21,0.164089,0.794051,27,0,0,5.0,0
4,crossplane,10,False,False,lower,50,15554,525,True,14,False,0.0,2957,1163,31,0.0,False,morning_06_11,Thursday,1,1.0,0,0,1,1,github,0,0,1,other_or_unknown,15,1,0,2,0,0,0,0,"0.4.5,0.4.7,0.4.5,0.4.7,0.4.8,0.4.8,0.3.0,0.3....",2017-10-24 01:51:55.687081 UTC,2022-09-22 09:51:50.87393 UTC,[],[aluttik@gmail.com],[],[],[Programming Language :: Python :: Implementat...,[],0,582450.0,2658719.0,NaN,"Apache License, Version 2.0, Apache 2.0",Reliable and fast NGINX configuration file par...,\n![Crossplane Logo](https://raw.githubusercon...,8,7,0.044565,0.798500,0,0,0,NaN,0


In [39]:
import re
def clean_package_name_re(name):
    # 1. Lowercase the name for consistency
    name = name.lower()
    
    # 2. Use regex to remove all characters that are NOT a letter (a-z) or a number (0-9)
    # The pattern [^a-z0-9] matches any character that is NOT in the set.
    cleaned_name = re.sub(r'[^a-z0-9]', '', name)
    
    return cleaned_name

In [40]:
packs = pd.read_csv("pack_downloads.csv")
high_download_mask = packs['total_downloads'] >= 1_000_000_000
high_download_packs = packs[high_download_mask]
top_legit_packages = high_download_packs['package_name'].apply(clean_package_name_re)
top_legit_packages.to_csv("top_legit_packages.csv", index=False)

"latest_description"

"latest_summary"

"licenses"

"latest_dependencies"

"distinct_classifiers"

"distinct_keywords"

"distinct_maintainers"

"distinct_authors"

"latest_project_urls"

"t_last_release"

"t_first_release"

"versions"



In [43]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import WhitespaceSplit

file_path = "top_legit_packages.csv"

# 1. Initialize the BPE model
# We set an UNK token, though it won't be heavily used in training.
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# 2. Set the pre-tokenizer
# Since package names are usually one word, we use a simple pre-tokenizer 
# that treats the entire name as a single unit, which is optimal for character-level BPE.
# If your package names were separated by spaces, you'd use Whitespace().
tokenizer.pre_tokenizer = WhitespaceSplit()

# 3. Configure the Trainer
trainer = BpeTrainer(
    # Set a small vocabulary size to force the discovery of common sub-words (brands)
    # The initial characters count as part of this size.
    vocab_size=300, 
    min_frequency=2, # Only merge pairs that appear at least twice
    special_tokens=["[UNK]"],
)

# 4. Train the tokenizer
print("Starting BPE training...")
files = [file_path]
tokenizer.train(files, trainer=trainer)
print("Training complete.")

Starting BPE training...



Training complete.


In [44]:
# Get the full vocabulary map {token: id}
vocab = tokenizer.get_vocab()

# Filter for the learned sub-words (merged tokens)
learned_subwords = []
for token in vocab.keys():
    # We are interested in tokens that are:
    # 1. Longer than one character (to exclude basic letters)
    # 2. Not a special token (like '[UNK]')
    if len(token) > 1 and token not in trainer.special_tokens:
        learned_subwords.append(token)

print("\n--- Top Learned Brand Sub-words ---")
# Sort the results, perhaps by length or just alphabetically
learned_subwords.sort(key=len, reverse=True) 

# Display the top results
for subword in learned_subwords[:20]:
    # To see its ID (which is roughly related to the order it was merged):
    # print(f"{subword} (ID: {vocab[subword]})")
    print(subword)


--- Top Learned Brand Sub-words ---
opentelemetryexporterotlpproto
opentelemetryexporterotlp
opentelemetryexpor
opentelemetry
googlecloud
jsonschema
googleauth
googleapi
opentelem
jsonsche
requests
request
extensi
terotlp
jupyter
crypto
client
common
poetry
typing


# The Pipeline Starts Here !!!!!!!!

In [ ]:
import ast
import math
from typing import List, Iterable, Tuple

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -------------------------------------------------------
# Config
# -------------------------------------------------------
INPUT_PATH = "pyspam_features.jsonl"
OUTPUT_PATH = "pyspam_features_with_offline.jsonl"

# Example reference sets (you should replace / extend these)
TOP_LEGIT_PACKAGES = [
    "requests", "numpy", "pandas", "flask", "django",
    "scipy", "pytest", "matplotlib", "scikit-learn"
]

BRAND_ALIASES = [
    "google", "microsoft", "amazon", "facebook", "meta",
    "apple", "paypal", "stripe", "github", "gitlab"
]

TOP_BRAND_PKGS = [
    "django", "flask", "requests", "tensorflow", "torch",
    "boto3", "azure", "google-cloud-storage", "stripe"
]

# Threshold for "typosquatting" closeness
LEV_THRESHOLD = 2

# Threshold for "low download" packages (you can tune this)
LOW_DOWNLOAD_THRESHOLD_30D = 50


# -------------------------------------------------------
# Utility: safe parsing of list-like fields from CSV/JSON
# -------------------------------------------------------
def parse_list_column(val):
    """
    Handle cases where a column is:
    - already a list (when loading from JSON)
    - a string representation of a Python list
    - NULL/NaN
    """
    if isinstance(val, list):
        return val
    if pd.isna(val):
        return []
    text = str(val)
    try:
        parsed = ast.literal_eval(text)
        if isinstance(parsed, list):
            return parsed
        return [parsed]
    except Exception:
        # fallback: treat as a single string
        return [text]


# -------------------------------------------------------
# Utility: extract base package name from requires_dist
# -------------------------------------------------------
def extract_pkg_name_from_requirement(req: str) -> str:
    """
    Roughly extract the package name from a requirement string.
    Examples:
      'requests>=2.0' -> 'requests'
      'google-cloud-storage==1.0; python_version<"3.10"' -> 'google-cloud-storage'
    """
    if not isinstance(req, str):
        return ""
    # Split on semicolon to drop markers, then on space and version operators
    base = req.split(";")[0].strip()
    # Remove version specifiers like <, >, =, !, ~
    # We just take the token until we hit one of those chars.
    for i, ch in enumerate(base):
        if ch in "<>!=~ ":
            base = base[:i]
            break
    return base.strip()


# -------------------------------------------------------
# Utility: Levenshtein distance (pure Python)
# -------------------------------------------------------
def levenshtein(a: str, b: str) -> int:
    """
    Simple dynamic programming Levenshtein distance implementation.
    """
    if a == b:
        return 0
    if len(a) == 0:
        return len(b)
    if len(b) == 0:
        return len(a)

    prev_row = list(range(len(b) + 1))
    for i, ca in enumerate(a, start=1):
        curr_row = [i]
        for j, cb in enumerate(b, start=1):
            insert_cost = curr_row[j - 1] + 1
            delete_cost = prev_row[j] + 1
            replace_cost = prev_row[j - 1] + (ca != cb)
            curr_row.append(min(insert_cost, delete_cost, replace_cost))
        prev_row = curr_row
    return prev_row[-1]


def min_levenshtein_to_set(s: str, ref: Iterable[str]) -> int:
    s = s or ""
    if not ref:
        return math.inf
    s_lower = s.lower()
    return min(levenshtein(s_lower, r.lower()) for r in ref)


def min_levenshtein_to_set_with_threshold(s: str, ref: Iterable[str], threshold: int) -> Tuple[int, bool]:
    """Return (distance, is_close_flag)."""
    dist = min_levenshtein_to_set(s, ref)
    return dist, dist <= threshold


# -------------------------------------------------------
# Step 1: Load data
# -------------------------------------------------------
df = pd.read_json(INPUT_PATH, lines=True)

# If some of these columns are stored as strings, we may want to ensure types:
list_like_cols = [
    "distinct_authors",
    "distinct_maintainers",
    "distinct_keywords",
    "distinct_classifiers",
    "latest_project_urls",
    "latest_dependencies",
]
for col in list_like_cols:
    if col in df.columns:
        df[col] = df[col].apply(parse_list_column)

# Ensure numeric booleans are numeric (BigQuery often exports as 0/1 integers or 'true'/'false')
bool_like_cols = [
    "has_homepage", "has_repo_url", "has_issue_tracker", "has_docs_url",
    "has_license", "has_prog_lang_classifier", "has_typing_classifier",
    "has_extras", "has_missing_author", "has_disposable_email"
]
for col in bool_like_cols:
    if col in df.columns:
        # convert 'true'/'false' strings to 1/0, etc.
        df[col] = df[col].astype(str).str.lower().map({"true": 1, "false": 0}).fillna(df[col]).astype(float)


# ---- Ensure is_spam is numeric and build legit mask ----
if "is_spam" in df.columns:
    df["is_spam"] = pd.to_numeric(df["is_spam"], errors="coerce")
    legit_mask = (df["is_spam"] == 0)
else:
    legit_mask = pd.Series([True] * len(df))

# Convert to NumPy array for indexing sparse matrices
legit_mask_np = legit_mask.to_numpy()

# -------------------------------------------------------
# Step 2: A.* name-based offline features
# -------------------------------------------------------
name_text = df["pkg_name"].astype(str).fillna("")

name_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 5))
X_name = name_vectorizer.fit_transform(name_text)

if legit_mask_np.sum() > 0:
    legit_centroid = X_name[legit_mask_np].mean(axis=0)
else:
    legit_centroid = X_name.mean(axis=0)

legit_centroid = np.asarray(legit_centroid)

sim_to_legit_name = cosine_similarity(X_name, legit_centroid).ravel()

df["n_lev_dist_to_top1"] = [
    min_levenshtein_to_set(name, TOP_LEGIT_PACKAGES)
    for name in df["pkg_name"].astype(str)
]

df["n_lev_dist_to_alias"] = [
    min_levenshtein_to_set(name, BRAND_ALIASES)
    for name in df["pkg_name"].astype(str)
]

df["sim_tfidf_to_legit_centroid"] = sim_to_legit_name

# -------------------------------------------------------
# Step 3: B.* description embedding distance offline
# -------------------------------------------------------
desc_text = df["latest_description"].fillna("").astype(str)

desc_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english"
)
X_desc = desc_vectorizer.fit_transform(desc_text)

if legit_mask_np.sum() > 0:
    legit_desc_centroid = X_desc[legit_mask_np].mean(axis=0)
else:
    legit_desc_centroid = X_desc.mean(axis=0)

legit_desc_centroid = np.asarray(legit_desc_centroid)
sim_to_legit_desc = cosine_similarity(X_desc, legit_desc_centroid).ravel()
df["dist_embed_to_legit_desc"] = 1.0 - sim_to_legit_desc

# -------------------------------------------------------
# Step 4: D.* maintainer-based offline features
#   - n_pkgs_by_maintainers_30d
#   - n_low_download_pkgs_by_maintainers
# -------------------------------------------------------
# Explode maintainers to build per-maintainer stats
maintainers_df = df[["pkg_name", "distinct_authors", "distinct_maintainers",
                     "t_age_last_release_days", "n_downloads_30d"]].copy()

# unify all maintainer emails for this stage
def collect_maintainers(row):
    return list(set(row["distinct_authors"] + row["distinct_maintainers"]))

maintainers_df["all_maintainers"] = maintainers_df.apply(collect_maintainers, axis=1)
maintains_exploded = maintainers_df.explode("all_maintainers")
maintains_exploded = maintains_exploded.rename(columns={"all_maintainers": "maintainer_email"})
maintains_exploded = maintains_exploded[maintains_exploded["maintainer_email"] != ""]

# a) n_pkgs_by_maintainers_30d (per maintainer)
recent_mask = maintains_exploded["t_age_last_release_days"] <= 30
maintainer_recent_pkg_counts = (
    maintains_exploded[recent_mask]
    .groupby("maintainer_email")["pkg_name"]
    .nunique()
    .rename("maintainer_recent_pkg_count")
    .reset_index()
)

# b) n_low_download_pkgs_by_maintainers
low_dl_mask = maintains_exploded["n_downloads_30d"].fillna(0) < LOW_DOWNLOAD_THRESHOLD_30D
maintainer_lowdl_pkg_counts = (
    maintains_exploded[low_dl_mask]
    .groupby("maintainer_email")["pkg_name"]
    .nunique()
    .rename("maintainer_low_download_pkg_count")
    .reset_index()
)

# join stats back onto exploded maintainer table
maintains_exploded = maintains_exploded.merge(
    maintainer_recent_pkg_counts, on="maintainer_email", how="left"
).merge(
    maintainer_lowdl_pkg_counts, on="maintainer_email", how="left"
)

maintains_exploded["maintainer_recent_pkg_count"] = maintains_exploded["maintainer_recent_pkg_count"].fillna(0)
maintains_exploded["maintainer_low_download_pkg_count"] = maintains_exploded["maintainer_low_download_pkg_count"].fillna(0)

# aggregate per package (e.g., sum or max over maintainers)
maintainer_agg = (
    maintains_exploded
    .groupby("pkg_name")
    .agg(
        n_pkgs_by_maintainers_30d=("maintainer_recent_pkg_count", "sum"),
        n_low_download_pkgs_by_maintainers=("maintainer_low_download_pkg_count", "sum"),
    )
    .reset_index()
)

# merge back to main df
df = df.merge(maintainer_agg, on="pkg_name", how="left")
df["n_pkgs_by_maintainers_30d"] = df["n_pkgs_by_maintainers_30d"].fillna(0)
df["n_low_download_pkgs_by_maintainers"] = df["n_low_download_pkgs_by_maintainers"].fillna(0)
df['n_latest_project_urls'] = df['latest_project_urls'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# -------------------------------------------------------
# Step 5: H.* dependency-based offline features
#   - has_dependency_to_top_brand
#   - has_dependency_lev_close_to_brand
# -------------------------------------------------------
def deps_base_names(deps: List[str]) -> List[str]:
    return [extract_pkg_name_from_requirement(d) for d in deps if isinstance(d, str)]


dep_base_names = df["latest_dependencies"].apply(deps_base_names)

# has_dependency_to_top_brand
df["has_dependency_to_top_brand"] = [
    int(any(base.lower() in {b.lower() for b in TOP_BRAND_PKGS} for base in bases))
    for bases in dep_base_names
]

# has_dependency_lev_close_to_brand
brand_set_lower = [b.lower() for b in TOP_BRAND_PKGS + BRAND_ALIASES]

close_flags = []
min_dists = []
for bases in dep_base_names:
    if not bases:
        min_dists.append(math.inf)
        close_flags.append(0)
        continue
    # compute minimum distance across all deps to any brand
    dists = [
        min_levenshtein_to_set(base, brand_set_lower)
        for base in bases
        if base
    ]
    if not dists:
        min_dists.append(math.inf)
        close_flags.append(0)
    else:
        min_dist = min(dists)
        min_dists.append(min_dist)
        close_flags.append(int(min_dist <= LEV_THRESHOLD))

df["min_dep_lev_to_brand"] = [d if d != math.inf else np.nan for d in min_dists]
df["has_dependency_lev_close_to_brand"] = close_flags


# ------------------------------------------------------
# Step 6: Time offline features
#   - t_time_of_day (morning, evening, afternoon, night)
# ------------------------------------------------------


def bucket_to_time_of_day(bucket):
    if 'evening' in bucket:
        return 'evening'
    elif 'morning' in bucket:
        return 'morning'
    elif 'night' in bucket:
        return 'night'
    elif "afternoon" in bucket:
        return 'afternoon'
    else:
        return 'unknown'

df['t_time_of_day'] = df["t_time_of_day_bucket"].apply(bucket_to_time_of_day)

# -------------------------------------------------------
# Step 7: Remove redundant features
# -------------------------------------------------------

df.drop(columns=["latest_description",

                "latest_summary",

                "licenses",

                "latest_dependencies",

                "distinct_classifiers",

                "distinct_keywords",

                "distinct_maintainers",

                "distinct_authors",

                "latest_project_urls",

                "t_last_release",

                "t_first_release",
                
                "t_time_of_day_bucket",
                
                "versions"], inplace=True)


# -----------------------------------------------------------
# Step 8: Fill Null
# -----------------------------------------------------------

# Numerical replacements
num_fill_values = {
    "t_median_release_gap_days": 0,
    "n_downloads_7d": 0,
    "n_downloads_30d": 0,
    "n_dependents_est": 0,
    "min_dep_lev_to_brand": 20,
}

for col, fill_value in num_fill_values.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(fill_value)

# Categorical: cat_license_family
if "cat_license_family" in df.columns:
    df["cat_license_family"] = (
        df["cat_license_family"]
        .fillna("unknown")
        .replace({"other_or_unknown": "unknown"})
    )

# -------------------------------------------------------
# Step 9: Save enriched dataset
# -------------------------------------------------------
df.to_json(OUTPUT_PATH, orient="records", lines=True)
print(f"Saved enriched features to {OUTPUT_PATH}")

Saved enriched features to pyspam_features_with_offline.jsonl


In [104]:
df.columns

Index(['pkg_name', 'n_name_len', 'has_digit_in_name', 'has_dash_or_underscore',
       'cat_name_case', 'n_summary_len', 'n_desc_len', 'n_desc_lines',
       'has_code_block_in_desc', 'n_urls_in_desc', 'has_suspicious_kw',
       'pct_non_ascii_desc', 't_age_first_release_days',
       't_age_last_release_days', 'n_versions', 't_median_release_gap_days',
       'has_single_release', 'cat_weekday_of_last_release', 'n_maintainers',
       'pct_free_email_domains', 'has_disposable_email', 'has_missing_author',
       'has_homepage', 'has_repo_url', 'cat_repo_host', 'has_issue_tracker',
       'has_docs_url', 'has_license', 'cat_license_family', 'n_classifiers',
       'has_prog_lang_classifier', 'has_typing_classifier', 'n_distributions',
       'n_requires', 'has_extras', 'rule_no_repo_low_desc_len',
       'rule_suspicious_name_and_dep', 'is_spam', 'n_downloads_7d',
       'n_downloads_30d', 'n_dependents_est', 'n_lev_dist_to_top1',
       'n_lev_dist_to_alias', 'sim_tfidf_to_legit_cent

# Model Training Starts Here !!!!!!!

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier


# ----------------------------------------------------
# Identify label
# ----------------------------------------------------
y = df["is_spam"].astype(int)

# ----------------------------------------------------
# Select features
# Exclude non-usable columns: text, lists, identifiers
# ----------------------------------------------------
drop_cols = [
    "pkg_name", 
    "is_spam"
]

X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

# ----------------------------------------------------
# Detect categorical & numerical columns
# ----------------------------------------------------
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

# ----------------------------------------------------
# Preprocessing
# ----------------------------------------------------
# dummy encoding the dataset

preprocessor = ColumnTransformer(
    transformers= [
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), 
         categorical_cols)
    ],
    remainder='passthrough'
)



# ----------------------------------------------------
# Random Forest model + pipeline
# ----------------------------------------------------

pipe = Pipeline([
    ('prep', preprocessor),
    ('cls', RandomForestClassifier(
        random_state=42,
        class_weight="balanced"   # Helps with imbalance
    ))  
])


# ----------------------------------------------------
# Parameter tuning (two parameters)
# ----------------------------------------------------
param_grid = {
    "cls__n_estimators": [150, 300, 500],
    "cls__max_depth": [None, 10, 20]
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=2
)

# ----------------------------------------------------
# Split data
# ----------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ----------------------------------------------------
# Train
# ----------------------------------------------------
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

# ----------------------------------------------------
# Evaluate on test set
# ----------------------------------------------------
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

print("\n=== Test ROC-AUC ===")
print(roc_auc_score(y_test, y_prob))

Categorical columns: ['cat_name_case', 'cat_weekday_of_last_release', 'cat_repo_host', 'cat_license_family', 't_time_of_day']
Numerical columns: ['n_name_len', 'n_summary_len', 'n_desc_len', 'n_desc_lines', 'n_urls_in_desc', 'pct_non_ascii_desc', 't_age_first_release_days', 't_age_last_release_days', 'n_versions', 't_median_release_gap_days', 'n_maintainers', 'pct_free_email_domains', 'has_disposable_email', 'has_missing_author', 'has_homepage', 'has_repo_url', 'has_issue_tracker', 'has_docs_url', 'has_license', 'n_classifiers', 'has_prog_lang_classifier', 'has_typing_classifier', 'n_distributions', 'n_requires', 'has_extras', 'rule_no_repo_low_desc_len', 'rule_suspicious_name_and_dep', 'n_downloads_7d', 'n_downloads_30d', 'n_dependents_est', 'n_lev_dist_to_top1', 'n_lev_dist_to_alias', 'sim_tfidf_to_legit_centroid', 'dist_embed_to_legit_desc', 'n_pkgs_by_maintainers_30d', 'n_low_download_pkgs_by_maintainers', 'n_latest_project_urls', 'has_dependency_to_top_brand', 'min_dep_lev_to_bran

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[CV] END .........cls__max_depth=None, cls__n_estimators=150; total time=   0.8s
[CV] END .........cls__max_depth=None, cls__n_estimators=150; total time=   0.9s
[CV] END .........cls__max_depth=None, cls__n_estimators=150; total time=   0.9s
[CV] END .........cls__max_depth=None, cls__n_estimators=150; total time=   0.9s
[CV] END .........cls__max_depth=None, cls__n_estimators=150; total time=   0.9s


/Users/reezhan/anaconda3/envs/finance/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END .........cls__max_depth=None, cls__n_estimators=300; total time=   1.6s
[CV] END .........cls__max_depth=None, cls__n_estimators=300; total time=   1.6s
[CV] END .........cls__max_depth=None, cls__n_estimators=300; total time=   1.7s


...

[CV] END ...........cls__max_depth=10, cls__n_estimators=500; total time=   3.1s
[CV] END ...........cls__max_depth=10, cls__n_estimators=500; total time=   3.1s
[CV] END ...........cls__max_depth=20, cls__n_estimators=300; total time=   1.9s
[CV] END ...........cls__max_depth=20, cls__n_estimators=300; total time=   1.8s
[CV] END ...........cls__max_depth=20, cls__n_estimators=300; total time=   1.9s


/Users/reezhan/anaconda3/envs/finance/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END ...........cls__max_depth=20, cls__n_estimators=300; total time=   2.0s
[CV] END ...........cls__max_depth=20, cls__n_estimators=300; total time=   1.9s
[CV] END ...........cls__max_depth=20, cls__n_estimators=500; total time=   2.7s
[CV] END ...........cls__max_depth=20, cls__n_estimators=500; total time=   2.3s
[CV] END ...........cls__max_depth=20, cls__n_estimators=500; total time=   2.0s
[CV] END ...........cls__max_depth=20, cls__n_estimators=500; total time=   2.0s


/Users/reezhan/anaconda3/envs/finance/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END ...........cls__max_depth=20, cls__n_estimators=500; total time=   1.9s
Best Params: {'cls__max_depth': None, 'cls__n_estimators': 150}
Best CV Score: 0.9999978940976902

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       945
           1       1.00      1.00      1.00      1806

    accuracy                           1.00      2751
   macro avg       1.00      1.00      1.00      2751
weighted avg       1.00      1.00      1.00      2751


=== Confusion Matrix ===
[[ 944    1]
 [   0 1806]]

=== Test ROC-AUC ===
1.0


In [106]:
MODEL_FEATURE_COLUMNS = ['n_name_len', 'has_digit_in_name', 'has_dash_or_underscore',
       'cat_name_case', 'n_summary_len', 'n_desc_len', 'n_desc_lines',
       'has_code_block_in_desc', 'n_urls_in_desc', 'has_suspicious_kw',
       'pct_non_ascii_desc', 't_age_first_release_days',
       't_age_last_release_days', 'n_versions', 't_median_release_gap_days',
       'has_single_release', 'cat_weekday_of_last_release', 'n_maintainers',
       'pct_free_email_domains', 'has_disposable_email', 'has_missing_author',
       'has_homepage', 'has_repo_url', 'cat_repo_host', 'has_issue_tracker',
       'has_docs_url', 'has_license', 'cat_license_family', 'n_classifiers',
       'has_prog_lang_classifier', 'has_typing_classifier', 'n_distributions',
       'n_requires', 'has_extras', 'rule_no_repo_low_desc_len',
       'rule_suspicious_name_and_dep', 'n_downloads_7d',
       'n_downloads_30d', 'n_dependents_est', 'n_lev_dist_to_top1',
       'n_lev_dist_to_alias', 'sim_tfidf_to_legit_centroid',
       'dist_embed_to_legit_desc', 'n_pkgs_by_maintainers_30d',
       'n_low_download_pkgs_by_maintainers', 'n_latest_project_urls',
       'has_dependency_to_top_brand', 'min_dep_lev_to_brand',
       'has_dependency_lev_close_to_brand', 't_time_of_day']

In [113]:
X.head()

,n_name_len,has_digit_in_name,has_dash_or_underscore,cat_name_case,n_summary_len,n_desc_len,n_desc_lines,has_code_block_in_desc,n_urls_in_desc,has_suspicious_kw,pct_non_ascii_desc,t_age_first_release_days,t_age_last_release_days,n_versions,t_median_release_gap_days,has_single_release,cat_weekday_of_last_release,n_maintainers,pct_free_email_domains,has_disposable_email,has_missing_author,has_homepage,has_repo_url,cat_repo_host,has_issue_tracker,has_docs_url,has_license,cat_license_family,n_classifiers,has_prog_lang_classifier,has_typing_classifier,n_distributions,n_requires,has_extras,rule_no_repo_low_desc_len,rule_suspicious_name_and_dep,n_downloads_7d,n_downloads_30d,n_dependents_est,n_lev_dist_to_top1,n_lev_dist_to_alias,sim_tfidf_to_legit_centroid,dist_embed_to_legit_desc,n_pkgs_by_maintainers_30d,n_low_download_pkgs_by_maintainers,n_latest_project_urls,has_dependency_to_top_brand,min_dep_lev_to_brand,has_dependency_lev_close_to_brand,t_time_of_day
0,11,False,False,lower,0,0,1,False,0,False,0.0,1119,1119,1,0.0,True,Saturday,0,0.0,0.0,0.0,0.0,0.0,unknown,0.0,0.0,0.0,unknown,0,0.0,0.0,2,0,0.0,1,0,0.0,0.0,0.0,8,8,0.014934,1.000000,0.0,0.0,0,0,20.0,0,evening
1,12,False,True,lower,59,0,1,False,0,False,0.0,1101,15,16,0.0,False,Thursday,1,0.0,0.0,0.0,1.0,1.0,github,0.0,0.0,1.0,unknown,4,1.0,0.0,12,8,1.0,0,0,367475.0,1878537.0,3.0,10,9,0.069909,1.000000,1.0,0.0,0,1,0.0,1,afternoon
2,23,False,True,lower,66,19482,489,True,11,False,0.0,2949,388,23,0.0,False,Tuesday,1,0.0,0.0,0.0,1.0,1.0,github,0.0,0.0,1.0,MIT,10,1.0,0.0,2,4,0.0,0,0,1714703.0,7594735.0,2.0,17,19,0.153751,0.842534,18.0,0.0,0,0,5.0,0,night
3,27,False,True,lower,70,13658,294,True,12,False,0.0,3570,617,16,0.0,False,Thursday,2,0.0,0.0,0.0,1.0,1.0,github,0.0,0.0,1.0,MIT,10,1.0,0.0,2,3,0.0,0,0,342109.0,1511511.0,1.0,21,21,0.164089,0.794051,27.0,0.0,0,0,5.0,0,morning
4,10,False,False,lower,50,15554,525,True,14,False,0.0,2957,1163,31,0.0,False,Thursday,1,1.0,0.0,0.0,1.0,1.0,github,0.0,0.0,1.0,unknown,15,1.0,0.0,2,0,0.0,0,0,582450.0,2658719.0,0.0,8,7,0.044565,0.798500,0.0,0.0,0,0,20.0,0,morning


In [1]:
F = df.set_index("pkg_name")
M = F.loc['pandas'][MODEL_FEATURE_COLUMNS]
best_model.predict_proba(M)

NameError: name 'df' is not defined

In [90]:
col = X.columns.to_frame().to_numpy()

In [91]:
fi = best_model.feature_importances_.reshape(-1,1)

In [96]:
feature_importances = pd.DataFrame(np.concatenate([col, fi], axis=1))
feature_importances.sort_values(by=1, ascending=False, inplace=True)
pd.set_option('display.max_rows', None)
feature_importances

,0,1
33,n_downloads_30d,0.19149
32,n_downloads_7d,0.165727
12,n_versions,0.106434
14,has_single_release,0.085788
60,cat_repo_host_unknown,0.069402
27,n_distributions,0.050466
10,t_age_first_release_days,0.044769
24,n_classifiers,0.041707
40,n_low_download_pkgs_by_maintainers,0.038757
20,has_repo_url,0.036575


numerical columns:

- t_median_release_gap_days nan -> 0

- n_downloads_7d nan -> 0

- n_downloads_30d nan -> 0

- n_dependents_est nan -> 0

- min_dep_lev_to_brand nan -> 20


Categorical Columns:

- cat_license_family   other_or_unknown -> unknown


In [103]:
import joblib

# Define the filename for your saved model
filename = 'random_forest_model.joblib'

# Save the model to the specified file
joblib.dump(best_model, filename)
print(f"Model saved to {filename}")

Model saved to random_forest_model.joblib


Breakdown:
	1.	User journey map (what the user experiences, step by step)
	2.	Pages / UI structure
	3.	Backend logic & data flow (incl. cleaned offline dataset + .joblib model)
	4.	quick Streamlit implement pattern

⸻

1. User Journey Map

The main persona is: “Python dev or security analyst who wants to quickly check if a package looks suspicious.”

Journey A: Check a single package
	1.	Landing → “Check a package”
	•	User lands on the dashboard.
	•	Sees: short tagline (“Check if a PyPI package looks suspicious”), big search bar.
	•	Inputs: package_name (e.g. virtualenv), optional version (or “latest”).
	2.	Loading → Behind the scenes
	•	App validates the package name.
	•	Loads model from .joblib.
	•	Fetches feature row for that package (from BigQuery or a precomputed feature table).
	•	Runs prediction → returns:
	•	risk_score (probability),
	•	risk_label (Low/Med/High),
	•	key feature values & explanations.
	3.	Result view → “Is this package safe?”
	•	At the top:
	•	Package name + version
	•	Big colored badge: High Risk / Medium Risk / Low Risk
	•	Numeric score (e.g. 0.87) and short explanation (“Higher = more likely scam”).
	•	Below:
	•	Key signals: “Single release”, “Very new package”, “Weird URL patterns”, etc.
	•	Time series or tiny plots for release/download activity to make it intuitive.
	4.	Deep dive → “Why did it get this score?”
	•	User can expand:
	•	Feature table (Release & Activity features, Text features, etc.).
	•	Flagged anomalies vs. baseline (e.g., “Has 1 release vs median 12 for legit packages”).
	•	Optional: SHAP/feature importance bar chart.
	5.	Next step
	•	User decides:
	•	“Ok, safe enough” → closes.
	•	“Looks shady” → they may blacklist it in their own systems (you can later add export/JSON).

⸻

Journey B: Explore suspicious packages (for demo wow-factor)
	1.	User clicks “Explore suspicious packages” on the navbar.
	2.	They see a table:
	•	Columns: pkg_name, risk_score, downloads, age_last_release_days, has_single_release, etc.
	•	Filters: “Risk score > 0.8”, “At least 1000 downloads”, “Updated in last 30 days”.
	3.	Clicking a row takes them to the same detail page as Journey A.

This journey sells the model visually: “Here are concrete libraries we’d review.”

⸻

2. Page / UI Structure

You can keep it to 3 main screens:
	1.	Home / Check Package
	•	Search bar for pkg_name
	•	Maybe a “recently checked” list.
	•	Quick stats card:
	•	packages scored
	•	% flagged high-risk
	•	last update time of feature table/model.
	2.	Package Detail Page
Sections:
(A) Overview card (top)
	•	pkg_name, maybe version.
	•	Risk badge: HIGH / MEDIUM / LOW.
	•	Risk score (0–1 or 0–100).
	•	2–3 bullet “reasons”:
	•	“Single release”
	•	“Last release 2 days ago”
	•	“Description very short / has suspicious URL patterns / etc.”
(B) Activity & Release Timeline
	•	Mini chart: releases over time (dates on x-axis).
	•	Maybe a bar/line for downloads vs time if you use file_downloads features later.
(C) Feature Table / Evidence panel
	•	Tabbed view:
	•	Release & Activity (your current focus)
	•	t_age_first_release_days
	•	t_age_last_release_days
	•	n_versions
	•	t_median_release_gap_days
	•	has_single_release
	•	t_time_of_day_bucket
	•	cat_weekday_of_last_release
	•	Other categories (Description, Repo, Author…) when you add them.
	•	For each feature, show:
	•	Value
	•	Typical range for legit packages (e.g. a tiny bar or “20th–80th percentile”)
	•	Indicator if it’s “out of normal range” (colored icon).
(D) Model Explanation (optional)
	•	Bar chart: top 5 features contributing most to the score.
	•	Text: “This package is flagged mostly because it has…”.
	3.	Explore / Rankings Page (nice to have, great for demos)
	•	Table of many packages with:
	•	risk_score, risk_label, total_downloads, age_last_release_days, etc.
	•	Sort and filter controls.
	•	Click → detail page.

⸻

3. Backend Logic & Data Flow

Already have a trained model saved as .joblib. Backend should do:

3.1 Core objects
	•	Model artifact:
	•	model.joblib – includes both the classifier and (ideally) the preprocessing/feature pipeline.
	•	finding features with 

3.2 Request → Response flow for “Check a package”

From a high level:
	1.	Input validation
	•	Receive pkg_name (and optionally version).
	•	Normalize (lowercase, trim).
	2.	Feature retrieval
Two options:
Option A: Precomputed features (strongly recommended)
	•	You run a daily/weekly BigQuery job that materializes all features into a table like:
	•	pypi_features.package_features
	•	Backend query:

SELECT *
FROM `your_project.pypi_features.package_features`
WHERE pkg_name = @pkg_name
LIMIT 1;


	•	This is fast and cheap for the dashboard.
Option B: On-the-fly feature computation per request
	•	For a live demo it’s acceptable:
	•	Query BigQuery pypi tables,
	•	Compute Release & Activity features in your backend code,
	•	Pass the resulting feature vector to the model.
	•	Slower & more complex, but flexible.

	3.	Feature vector assembly
	•	Map DB columns → model input order (same as during training).
	•	Ensure all required columns exist; fill missing with defaults/NaN-handling like you did in training.
	4.	Prediction
	•	Load model from .joblib (on startup, not per request).
	•	Call:
	•	proba = model.predict_proba([x])[0, 1]
	•	label = proba > threshold or map into {Low, Med, High}.
	•	Optionally: compute feature importance / explanation if you stored it.
	5.	Response building
	•	Return JSON object like:

{
  "pkg_name": "virtualenv",
  "risk_score": 0.87,
  "risk_label": "HIGH",
  "features": {
    "t_age_first_release_days": 6650,
    "t_age_last_release_days": 30,
    "n_versions": 244,
    "t_median_release_gap_days": 0,
    "has_single_release": false,
    "t_time_of_day_bucket": "morning_06_11",
    "cat_weekday_of_last_release": "Wednesday"
  },
  "top_signals": [
    "Frequent releases with very small median gap",
    "Long package history but unusual recent pattern"
  ]
}


	6.	Front-end rendering
	•	The dashboard reads this JSON and:
	•	Colors badges,
	•	Populates charts,
	•	Displays feature tables.

3.3 Backend endpoints (example, if you use FastAPI or similar)
	•	GET /api/predict?pkg_name=...
	•	Returns prediction + core features.
	•	GET /api/package/{pkg_name}
	•	Returns richer package info for the detail page (metadata, charts data, etc.).
	•	GET /api/ranking?min_score=0.8&limit=100
	•	Returns top suspicious packages for the Explore page.

⸻

4. Two Implementation Approaches

Approach 1: Single-file Streamlit app (fastest path for you)
	•	Streamlit app script:
	•	Loads model.joblib once.
	•	Uses google-cloud-bigquery to query your feature table or live features.
	•	UI:
	•	Text input for pkg_name.
	•	“Check package” button → calls backend code inside the same script.
	•	Displays cards, tables, charts with st.metric, st.table, st.altair_chart etc.
	•	Pros:
	•	Very fast to implement.
	•	Pure Python, minimal boilerplate.
	•	Cons:
	•	Less “production-y”, but perfect for research/demo.

Approach 2: FastAPI backend + React/Next.js frontend
	•	Backend (FastAPI):
	•	On startup: load .joblib model and BigQuery client.
	•	Implement endpoints GET /api/predict, /api/ranking, etc.
	•	Host on GCP (Cloud Run) or similar.
	•	Frontend:
	•	React app (or just a simple HTML template) that:
	•	Calls /api/predict when the user searches.
	•	Renders cards/charts.
	•	Pros:
	•	Closer to something you can hand to a security team.
	•	Clear separation of concerns.
	•	Cons:
	•	More code to write and maintain.

⸻

If you tell me which stack you want to start with (Streamlit vs FastAPI+frontend), I can sketch:
	•	A concrete API contract (exact JSON schema for /predict), and
	•	A wireframe or component breakdown for the dashboard (e.g., which Streamlit widgets or which React components).

In [5]:
import pandas as pd
df = pd.read_json('pyspam_features_with_offline.jsonl', lines = True)
df.set_index("pkg_name", inplace=True)

df[df['is_spam']== 1].head()

,n_name_len,has_digit_in_name,has_dash_or_underscore,cat_name_case,n_summary_len,n_desc_len,n_desc_lines,has_code_block_in_desc,n_urls_in_desc,has_suspicious_kw,...,n_lev_dist_to_alias,sim_tfidf_to_legit_centroid,dist_embed_to_legit_desc,n_pkgs_by_maintainers_30d,n_low_download_pkgs_by_maintainers,n_latest_project_urls,has_dependency_to_top_brand,min_dep_lev_to_brand,has_dependency_lev_close_to_brand,t_time_of_day
pkg_name,,,,,,,,,,,,,,,,,,,,,
aowdjpawojd,11,False,False,lower,0,0,1,False,0,False,...,8,0.014934,1.000000,0,0,0,0,20,0,evening
myshellcode3,12,True,False,lower,35,0,1,False,0,False,...,10,0.046855,1.000000,0,26,0,0,20,0,afternoon
zafira,6,False,False,lower,0,0,1,False,0,False,...,5,0.019379,1.000000,0,0,0,0,20,0,night
aml-ds-pipeline-contrib,23,False,True,lower,14,25,3,False,0,False,...,18,0.109135,0.990623,0,0,0,0,20,0,evening
14Cent,6,True,False,mixed,12,0,1,False,0,False,...,5,0.034022,1.000000,0,0,0,0,20,0,evening
